# Détection des fraudes à la carte de crédit #

## 1. Introduction
### 1.1. Introduction du projet
Ce projet consiste à construire un modèle de prédiction anti-fraude des cartes de crédit en utilisant les données historiques des transactions par carte de crédit, afin de détecter à l'avance le vol des cartes de crédit des clients.
### 1.2. Dataset
Nous utilisont le jeu de données sur la site https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud.
>The dataset contains transactions made by credit cards in September 2013 by European cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.
>
>It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-sensitive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.
### 1.3. Scénarios de données
Ce jeu de données est constitué des transactions par carte de crédit, la problématique est de prédire si le client sera victime de fraude à la carte de crédit. Il y a seulement deux situations : fraude et non fraude. Et comme les données sont déjà classifiés par la colonne "Class", il s'agit d'un scénario d'apprentissage supervisé. C'est la raison pour laquelle la prédiction de fraude à la carte de crédit est une problématique de classification binaire. 

## 2. Prétraitement des données ##

In [ ]:
!pip install imblearn

In [ ]:
# imports

# pandas
import pandas as pd

# numpy
import numpy as np

# math
import math

# seaborn
import seaborn as sns

# time
import time

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

# Visualisation des données
from plotly import subplots
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# sampling
from imblearn.pipeline import make_pipeline
from imblearn.pipeline import Pipeline
#over
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN
#under
from imblearn.under_sampling import RandomUnderSampler, NearMiss
#mix
from imblearn.combine import SMOTEENN, SMOTETomek

In [ ]:
df = pd.read_csv('creditcard.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isnull().any()

>Il n'y a pas de valeurs manquantes dans les données et aucun traitement des valeurs manquantes n'est nécessaire.

## 3. Feature Engineering
Le Feature Engineering est un processus qui consiste à transformer les données brutes en caractéristiques représentant plus précisément le problème sous-jacent au modèle prédictif. https://datascientest.com/feature-engineering

Comme les colonnes V1 à V28, les données sont transformés par PCA, on a pas besoin de faire la Feature Extraction. Par contre, les colonnes "Time" et "Amount" ont les types de données très différents par rapport les autres, il faut faire la **Feature scaling**. 

### 3.1. Feature Selection
Dans le jeu de données, il y a 30 variables. Si nous utilisont tous les varisables, nous risqueront de faire le sur-apprentissage. Pour éviter cette situation, nous ferons la Feature Selection.

In [ ]:
# Transformer la colonne "Time" en heure de la journée

df['Hour'] = df["Time"].apply(lambda x : divmod(x, 3600)[0])

In [ ]:
def convert_hour(x):
    if x >= 24 :
        return x -24
    else:
        return x

In [ ]:
df['Hour'] = df['Hour'].apply(lambda x : convert_hour(x))

In [ ]:
df.describe().T

In [ ]:
df_fraud = df.loc[df["Class"]==1]
df_nonfraud = df.loc[df["Class"]==0]

# regarder la correlation entre les variables
# Ici, on suppose que la relation est linéaire, la valeur est dans [-1,1]
# Si la valeur vaut 1, les variables sont en relation linéaire
corrs_nonfraud = df_nonfraud.loc[:, df.columns != "Class"].corr()
corrs_fraud = df_fraud.loc[:, df.columns != "Class"].corr()

# affichier seulement le triangle
mask_fraud = np.triu(np.ones_like(corrs_fraud, dtype=bool))
corrs_fraud = corrs_fraud.mask(mask_fraud)
mask_nonfraud = np.triu(np.ones_like(corrs_nonfraud, dtype=bool))
corrs_nonfraud = corrs_nonfraud.mask(mask_nonfraud)

In [ ]:
fig = px.imshow(
    corrs_nonfraud.to_numpy().round(2), 
    color_continuous_scale="RdBu_r",
    x=list(corrs_nonfraud.index.values),
    y=list(corrs_nonfraud.columns.values)
)
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title_text='Non Fraud', title_x=0.5
)
fig.show()

In [ ]:
fig = px.imshow(
    corrs_fraud.to_numpy().round(2), 
    color_continuous_scale="RdBu_r",
    x=list(corrs_fraud.index.values),
    y=list(corrs_fraud.columns.values)
)
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title_text='Fraud', title_x=0.5
)
fig.show()

#### 3.1.1. Correlation entre les variables
Nous recherchons la correlation entre les variables. Nous supposons que les relations suivent la loi normal.  
>Selon les figure en dessus, dans la situation de fraud, les corrélations entre certaines variables sont plus prononcées. La variation entre V1, V2, V3, V4, V5, V6, V7, V9, V10, V11, V12, V14, V16, V17, V18 et V19 montre un certain schéma dans l'échantillon d'écrémage de cartes de crédit.

In [ ]:
# relation entre les feutures et Class

def distplot(data):
    group_labels = ['fraud', 'non fraud']
    hist_data = [df[data][df["Class"]==1], df[data][df["Class"]==0]]
    fig = ff.create_distplot(hist_data, group_labels, bin_size=.5, show_rug=False)
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey', showline=True, linecolor='LightGrey', mirror=True)
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey', showline=True, linecolor='LightGrey', mirror=True)
    fig.update_layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)', 
        height=350, 
        title="histogramme de " + data,
        title_font_color="Grey",
    )
    fig.show()
    return

variables = df.iloc[:,1:29]
for variable in variables:
    distplot(variable) 

#### 3.1.2. Distribution des données
Nous regardons à nouveau la distribution des données. Nous préférons sélectionner des variables qui ont des distributions significativement différentes sous différentes classes. Comme le montre les images ci-dessus, nous voudrions supprimer les variables V8, V13, V15, V20, V21, V22, V23, V24, V25, V26, V27 et V28. Cela est également conforme à la conclusion à laquelle nous sommes parvenus au chapitre 3.1.1. Nous supprimons aussi la colonne "Time" et gardons la colonne "Hour" en considérant le niveau de dispersion

In [ ]:
list_drop = ['V8', 'V13', 'V15', 'V17', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Time']
df = df.drop(list_drop, axis=1)
df.shape

In [ ]:
df.describe().T

## 3.2. Feature Scaling
Par rapport aux autres colonnes, les caractéristiques des données des colonnes "Hour" et "Amount" sont très différentes. Nous utilisons l'approche de standardisation, qui maintient l'information utile contenue dans les valeurs aberrantes et rend l'algorithme moins affecté par les valeurs aberrantes. (voir le lien https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [ ]:
# Standardisation

col = ['Amount', 'Hour']
sc = StandardScaler()
df[col] = sc.fit_transform(df[col])
df.describe().T

## 4. Entraînement du modèle
### 4.1. Dataset "train" et "test"
Nous séparerons le jeu de données en  parties "entraînement", "évaluation" et "test" en adoptant la validation croisée pour éviter la situation sur-apprantisage.

In [ ]:
x_variables = list(df.columns)
x_variables.remove('Class') #supprimer la colonne cible "Class" pour x
X = df[x_variables] 
y = df["Class"]
n_echantillon = y.shape[0]
n_echantillon_pos = y[y==0].shape[0]
n_echantillon_neg = y[y==1].shape[0]
print('nombre d\'échantillons: {}; les échantillons positifs: {:.2%}; les échantillons négatifs: {:.2%}'.format(n_echantillon, n_echantillon_pos/n_echantillon, n_echantillon_neg/n_echantillon))
print('nb de variables : ', X.shape[1])

In [ ]:
# séparer le jeu de données en partie train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [ ]:
# Visualisation des données jusqu'à 2 dimensions en utilisant PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
X_pca = pd.DataFrame(X_pca)
X_pca.columns=["pca_a","pca_b"]
X_pca["y"] = y

In [ ]:
# affichage de la distribution de données

sns.set()
sns.lmplot(x="pca_a", y="pca_b",data=X_pca, hue="y", fit_reg=False, markers=["o","x"],height=8,aspect=1.5,legend=False)
plt.legend(fontsize=20,bbox_to_anchor=(0.98, 0.6),edgecolor ='r')   
plt.xlabel("axis_1",fontsize=17)
plt.ylabel("axis_2",fontsize=17)

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train)
X_pca = pd.DataFrame(X_pca)
X_pca.columns=["pca_a","pca_b"]
X_pca["y"] = y_train

In [ ]:
rand = RandomUnderSampler()
xx, yy = rand.fit_resample(X_train, y_train)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(xx)
X_pca = pd.DataFrame(X_pca)
X_pca.columns=["pca_a","pca_b"]
X_pca["y"] = yy

In [ ]:
yy

### 4.2. Régression logistique
#### 4.2.1. Traitement le déséquilibre de données
Pour la variable cible "Class", les valeurs sont très déséquilibrées, cela peut avoir un impact sur l'apprentissage des modèles. Alors nous allons tester les méthodes sampling afin de traiter le déséquilibre de l'échantillon. 

In [ ]:
# choisir les méthodes sampling
sampling_methods = [
    SMOTE(random_state=42), 
    BorderlineSMOTE(random_state=42, kind='borderline-1'), 
    ADASYN(random_state=42), 
    NearMiss(),
    RandomUnderSampler(),
    SMOTEENN(random_state=42, n_jobs=-1), 
    SMOTETomek(random_state=42, n_jobs=-1)
]

names = [
    'SMOTE', 
    'Borderline SMOTE', 
    'ADASYN', 
    'NearMiss',
    'RandomUnderSampler',
    'SMOTE+ENN', 
    'SMOTE+Tomek'
]

#### 4.2.2. Entraînement le modèle
Nous séparerons le jeu de données en 3 parts "entraînement", "évaluation" et "test" en adoptant la validation croisée pour éviter la situation sur-apprantisage. Le modèle peut apprendre sur l'ensemble d'entraînement, les paramètres sont réglés sur l'ensemble d'évaluation et, enfin, la performance du modèle est évaluée à l'aide des données de l'ensemble de test.

In [ ]:
def ensemble_method(method):
    count = 0
    xx, yy = method.fit_resample(X_train, y_train)
    y_pred, y_prob = np.zeros(len(X_test)), np.zeros(len(X_test))
    for X_ensemble, y_ensemble in zip(xx, yy):
        model = LogisticRegression()  
        model.fit(X_ensemble, y_ensemble)
        y_pred += model.predict(X_test)
        y_prob += model.predict_proba(X_test)[:, 1]
        count += 1
    return np.where(y_pred >= 0, 1, -1), y_prob/count

In [ ]:
# courbe ROC

plt.figure(figsize=(15,8))
for (name, method) in zip(names, sampling_methods):
    t0 = time.time()
    model = make_pipeline(method, LogisticRegression())  
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_prob, pos_label=1)
    plt.plot(fpr, tpr, lw=3, label='{} (AUC={:.2f}, time={:.2f}s)'.
             format(name, auc(fpr, tpr), time.time() - t0))
    plt.xlabel("FPR", fontsize=17)
    plt.ylabel("TPR", fontsize=17)
    plt.title("ROC Curve",fontsize=17)
    plt.legend(fontsize=14)

>Selon le résultat, les courbes ROC présentent une estimation excessivement optimiste de l'effet sauf la méthode NearMiss, du coup nous voudrions utiliser la coubre PR

In [ ]:
# courbe PR (Precision Recall) 

plt.figure(figsize=(15,8))
for (name, method) in zip(names, sampling_methods):
    t0 = time.time()
    model = make_pipeline(method, LogisticRegression())
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y_test, y_prob, pos_label=1)
    plt.plot(recall, precision, lw=3, label='{} (AUC={:.2f}, time={:.2f}s)'.
             format(name, auc(recall, precision), time.time() - t0))
    plt.xlabel("Recall", fontsize=17)
    plt.ylabel("Precision", fontsize=17)
    plt.title("PR Curve",fontsize=17)
    plt.legend(fontsize=14, loc="upper right")

>Selon la courbe PR, nous préférons choisir la méthode SMOTE+ENN

In [ ]:
class_names = ['non fraud', 'fraud']

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("------------------------Original---------------------- \n", 
      classification_report(y_test, y_pred, target_names=class_names), '\n')

model = make_pipeline(SMOTE(random_state=42), LogisticRegression())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("--------------------------SMOTE----------------------- \n",
      classification_report(y_test, y_pred, target_names=class_names), '\n')

model = make_pipeline(RandomUnderSampler(), LogisticRegression())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("-------------------RandomUnderSampler----------------- \n",
      classification_report(y_test, y_pred, target_names=class_names), '\n')

model = make_pipeline(SMOTEENN(random_state=42, n_jobs=-1), LogisticRegression())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("------------------------SMOTEENN---------------------- \n",
      classification_report(y_test, y_pred, target_names=class_names), '\n')

In [ ]:
# changer le seuil
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, random_state=0, stratify=y)


class_names = ['non fraud', 'fraud']

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("------------------------Original---------------------- \n", 
      classification_report(y_test, y_pred, target_names=class_names), '\n')

model = make_pipeline(SMOTE(random_state=42), LogisticRegression())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("--------------------------SMOTE----------------------- \n",
      classification_report(y_test, y_pred, target_names=class_names), '\n')

model = make_pipeline(SMOTEENN(random_state=42, n_jobs=-1), LogisticRegression())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("------------------------SMOTEENN---------------------- \n",
      classification_report(y_test, y_pred, target_names=class_names), '\n')

#### 4.2.2 Optimisation du modèle
Nous trouverons les meilleurs paramètres dans LogisticRegression

In [ ]:
# cross-validation avec GridSearchCV

model = Pipeline([
    ('sampling', SMOTE(random_state=42)),
    ('classification', LogisticRegression())
])

params = {
    'classification__C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

grid = GridSearchCV(model, params, n_jobs=-1, scoring='f1', cv=10)
grid.fit(X_train, y_train)
results = pd.DataFrame(grid.cv_results_) 
best = np.argmax(results.mean_test_score.values)
print("Best parameters: {}".format(grid.best_params_))
print("Best cross-validation score: {:.5f}".format(grid.best_score_))

In [ ]:
# cross-validation avec KFold

list_C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

def find_best_C(x_train, y_train):
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    kf = KFold(n_splits=10)
    best_C = 0
    best_f1 = 0
    for C in list_C:
        sum_f1 = 0
        nb_test = 0
        for i_train, i_test in kf.split(x_train):
            x_train_kf, x_test_kf = x_train[i_train], x_train[i_test]
            y_train_kf, y_test_kf = y_train[i_train], y_train[i_test]
            model = make_pipeline(SMOTE(random_state=42), LogisticRegression(C=C))
            model.fit(x_train_kf, y_train_kf)
            pred_y = model.predict(x_test_kf)
            sum_f1 += f1_score(y_test_kf, pred_y)
            nb_test += 1
        aveg_f1 = sum_f1/nb_test
        if aveg_f1 > best_f1:
            best_C = C
            best_f1 = aveg_f1
    return (best_C, best_f1)

In [ ]:
(best_C, best_f1) = find_best_C(X_train, y_train)
print('Best C : ', best_C)
print('Best f1 : ', best_f1)

In [ ]:
clf = LogisticRegression(C=1)
clf.fit(x_train, y_train)
prediction_test = clf.predict(x_test)
print("accuracy_test : {:.10f}".format(accuracy_score(prediction_test, y_test, normalize=True)))
print("recall_test : {:.10f}".format(recall_score(prediction_test, y_test)))
print("precision_test : {:.10f}".format(precision_score(prediction_test, y_test)))